<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ModelOps demo - Python H2O AutoML using Git
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

![image](images/git_meth.png) 

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
This notebook will cover the Operationalization of the PIMA diabetes use case with H2O model format. <b>H2O</b> is an open source, distributed in-memory machine learning library with linear scalability. H2O supports the most widely used statistical & machine learning algorithms including gradient boosted machines, generalized linear models, deep learning and more. The most used capabilities in H2O are the <b>AutoML</b> capabilities that provides.<br>In this example, we will use the AutoML capabilities to generate a H2O xgboost and operationalize it through ModelOps in the same Model Catalog than other trained models based on other libraries.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Steps in this Notebook</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
<li>Configure the Environment </li>
 <li>Connect to Vantage</li>
 <li>Define Training function</li>
 <li>Define Evaluate function</li>
 <li>Define Scoring function</li>
 <li>Define Model Metadata </li>
 <li>Commit and Push to Git to let ModelOps manage</li>
 <li>ModelOps full lifecycle till deployment</li>
 <li>ModelOps Monitoring</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Configure the Environment</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Libraries installation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>A restart of the Kernel is needed to confirm changes</b>. We use -q parameter for a non-verbose log of the installation command, you may remove this parameter if you want to know all the steps of the pip installation.</p>

In [ ]:
#%pip install -q teradatamodelops==7.0.3 h2o==3.44.0.3 matplotlib==3.8.2 install-jdk==1.0.4

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Hint:</b><i>The easy way to restart the kernel to bring the above installed software into memory is to type zero zero (<b> 0 0 </b>). </i></p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.2 Libraries import</b></p>

In [ ]:
from teradataml import *
import os
import getpass
import logging
import sys


<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=08_ModelOps_GIT_PIMA_Python_H2OAutoML.ipynb;' UPDATE FOR SESSION; ''')

# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"

# set the path to the local project repository for this model demo
model_local_path = '~/modelops-demo-models/model_definitions/pima_h2o_automl'
res = os.system(f'mkdir -p {model_local_path}/model_modules')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
#%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_cloud');"        # Takes 10 seconds
%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_local');"        # Takes 30 seconds

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Creating predictions and model table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create a predictions table where we get our model predictions and the model table where we will upload the model created.</p>

In [ ]:
#ddl for Aoa_Byom_Models 
query = '''CREATE SET TABLE Aoa_Byom_Models 
     (
      model_version VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_type VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      project_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      deployed_at TIMESTAMP(6) DEFAULT CURRENT_TIMESTAMP(6),
      model BLOB(2097088000))
UNIQUE PRIMARY INDEX ( model_version );
'''
 
try:
    execute_sql(query)
except:
    db_drop_table('Aoa_Byom_Models')
    execute_sql(query) 


In [ ]:
#ddl for Pima_Patient_Predictions
query = '''CREATE MULTISET TABLE Pima_Patient_Predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      PatientId BIGINT,
      HasDiabetes BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
'''

try:
    execute_sql(query)
except:
    db_drop_table('Pima_Patient_Predictions')
    execute_sql(query) 



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Define Training Function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The training function takes the following shape </p>

```python
def train(context: ModelContext, **kwargs):
    aoa_create_context()
    
    # your training code
    
    # export model artefacts
    mojo = model.download_mojo(path=context.artifact_output_path, get_genmodel_jar=True)
    new_mojo = os.path.join(os.path.abspath(os.getcwd()), context.artifact_output_path, "model.h2o")
    if os.path.isfile(new_mojo):
        print("The file already exists")
    else:
        # Rename the file
        os.rename(mojo, new_mojo)
    
    record_training_stats(...)
```
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can execute this from the CLI or directly within the notebook as shown.</p>

In [ ]:
%%writefile $model_local_path/model_modules/training.py
from teradataml import DataFrame
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import os
import h2o
from h2o.automl import H2OAutoML


def check_java():
    try:
        print(os.environ['JAVA_HOME'])
    except:
        print ('Installing Java...')
        import jdk
        jdk.install('17', path='/usr/local/jdk')
        os.environ['JAVA_HOME'] = '/usr/local/jdk/jdk-17.0.9+9'


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    
    # read training dataset from Teradata and convert to pandas
    check_java()
    h2o.init()
    train_df = DataFrame.from_query(context.dataset_info.sql)
    train_hdf = h2o.H2OFrame(train_df.to_pandas())

    # convert target column to categorical
    train_hdf[target_name] = train_hdf[target_name].asfactor()

    print("Starting training...")
  
    # Execute AutoML on training data
    aml = H2OAutoML(max_models=context.hyperparams['max_models'], seed=context.hyperparams['seed'])
    aml.train(x=feature_names, y=target_name, training_frame=train_hdf)

    # Here we are getting the best GBM algorithm for demo purposes
    model = aml.get_best_model(algorithm="gbm")
    if not model:
        model = aml.leader

    print("Finished training")

    # export model artefacts
    mojo = model.download_mojo(path=context.artifact_output_path, get_genmodel_jar=True)
    new_mojo = os.path.join(os.path.abspath(os.getcwd()), context.artifact_output_path, "model.h2o")
    if os.path.isfile(new_mojo):
        print("The file already exists")
    else:
        # Rename the file
        os.rename(mojo, new_mojo)

    print("Saved trained model")

    try:
        model.varimp_plot(server=True, save_plot_path=os.path.join(os.path.abspath(os.getcwd()), context.artifact_output_path, "feature_importance.png"))
        fi = model.varimp(True)
        fix = fi[['variable','scaled_importance']]
        fis = fix.to_dict('records')
        feature_importance = {v['variable']:float(v['scaled_importance']) for (k,v) in enumerate(fis)}
    except:
        print("Warning: This model doesn't support feature importance (Stacked Ensemble)")
        aml.varimp_heatmap()
        save_plot('Feature Heatmap', context=context)
        feature_importance = {}

    record_training_stats(train_df,
                          features=feature_names,
                          targets=[target_name],
                          categorical=[target_name],
                          feature_importance=feature_importance,
                          context=context)

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# Check Java installation, installs if required
try:
    print(f"Java is installed at {os.environ['JAVA_HOME']}")
except:
    os.environ['JAVA_HOME'] = '/home/jovyan/.jdk/jdk-17.0.9+9'
    if os.path.isdir(os.environ['JAVA_HOME']) is False:
        print ('Installing Java...')
        import jdk
        jdk.install('17', path='/home/jovyan/.jdk')

# define the training dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes
FROM Demo_ModelOps.PIMA_PATIENT_FEATURES F 
JOIN Demo_ModelOps.PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
"""

feature_metadata =  {
    "database": "demo_modelops",
    "table": "aoa_statistics_metadata"
}
hyperparams = {"max_models": 3, "seed": 1}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]

from aoa import ModelContext, DatasetInfo

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)


ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="artifacts/",
                   model_version="h2oaml_v1",
                   model_table="model_h2oaml_v1")

sys.path.append(os.path.expanduser(f"{model_local_path}/model_modules"))
import training
training.train(context=ctx)

In [ ]:
# Check the generated files
!ls -lh artifacts

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Define Evaluation Function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The evaluation function takes the following shape</p>

```python
def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    with open(f"{context.artifact_input_path}/model.h2o", "rb") as f:
        model_bytes = f.read()
    
    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)
    
    # your evaluation logic
    
    record_evaluation_stats(...)
```
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can execute this from the CLI or directly within the notebook as shown.</p>

In [ ]:
%%writefile $model_local_path/model_modules/evaluation.py
from sklearn import metrics
from teradataml import DataFrame, copy_to_sql, get_context, H2OPredict
from aoa import (
    record_evaluation_stats,
    save_plot,
    aoa_create_context,
    store_byom_tmp,
    ModelContext
)
import json
import os
import matplotlib


def evaluate(context: ModelContext, **kwargs):

    aoa_create_context()

    with open(f"{context.artifact_input_path}/model.h2o", "rb") as f:
        model_bytes = f.read()

    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)

    target_name = context.dataset_info.target_names[0]

    byom_target_sql = "CAST(prediction AS INT)"

    h2o = H2OPredict(
        modeldata=model,
        newdata=DataFrame.from_query(context.dataset_info.sql),
        accumulate=[context.dataset_info.entity_key, target_name]
    )

    predictions_df = h2o.result

    predictions_df.to_sql(table_name="predictions_tmp", if_exists="replace", temporary=True)

    metrics_df = DataFrame.from_query(f"""
    SELECT 
        {target_name} as y_test, 
        {byom_target_sql} as y_pred
        FROM predictions_tmp
    """)
    metrics_df = metrics_df.to_pandas()

    y_pred = metrics_df[["y_pred"]]
    y_test = metrics_df[["y_test"]]

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics.accuracy_score(y_test, y_pred)),
        'Recall': '{:.2f}'.format(metrics.recall_score(y_test, y_pred)),
        'Precision': '{:.2f}'.format(metrics.precision_score(y_test, y_pred)),
        'f1-score': '{:.2f}'.format(metrics.f1_score(y_test, y_pred))
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)

    cf = metrics.confusion_matrix(y_test, y_pred)
    display = metrics.ConfusionMatrixDisplay(confusion_matrix=cf)
    display.plot()
    save_plot('Confusion Matrix', context=context)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=context.model_version)
    display.plot()
    save_plot('ROC Curve', context=context)

    # calculate stats if training stats exist
    if os.path.exists(f"{context.artifact_input_path}/data_stats.json"):
        record_evaluation_stats(features_df=DataFrame.from_query(context.dataset_info.sql),
                                predicted_df=DataFrame("predictions_tmp"),
                                context=context)

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM Demo_ModelOps.PIMA_PATIENT_FEATURES F 
JOIN Demo_ModelOps.PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="artifacts/",
                   artifact_input_path="artifacts/",
                   model_version="h2oaml_v1",
                   model_table="model_h2oaml_v1")

# drop volatile table from session if executing multiple times
try:
    db_drop_table('byom_models_tmp')
except: 
    pass

import evaluation
evaluation.evaluate(context=ctx)

# view evaluation results
import json
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

In [ ]:
# Check the generated files
!ls -lh artifacts

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Define Scoring Function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The scoring function takes the following shape</p>

```python
def score(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    with open(f"{context.artifact_input_path}/model.h2o", "rb") as f:
        model_bytes = f.read()

    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)
    
    # your evaluation logic
    
    record_scoring_stats(...)
```

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can execute this from the CLI or directly within the notebook as shown.</p>

In [ ]:
%%writefile $model_local_path/model_modules/scoring.py
from teradataml import copy_to_sql, get_context, DataFrame, H2OPredict
from aoa import (
    record_scoring_stats,
    aoa_create_context,
    store_byom_tmp,
    ModelContext
)


def score(context: ModelContext, **kwargs):

    aoa_create_context()

    with open(f"{context.artifact_input_path}/model.h2o", "rb") as f:
        model_bytes = f.read()

    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)

    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    byom_target_sql = "CAST(prediction AS INT)"

    print("Scoring")
    h2o = H2OPredict(
        modeldata=model,
        newdata=DataFrame.from_query(context.dataset_info.sql),
        accumulate=context.dataset_info.entity_key)

    print("Finished Scoring")


    # store the predictions
    predictions_df = h2o.result
    
    # add job_id column so we know which execution this is from if appended to predictions table
    predictions_df = predictions_df.assign(job_id=context.job_id)
    cols = {}
    cols[target_name] = predictions_df['prediction']
    predictions_df = predictions_df.assign(**cols)
    predictions_df = predictions_df[["job_id", entity_key, target_name, "json_report"]]

    copy_to_sql(df=predictions_df,
                schema_name=context.dataset_info.predictions_database,
                table_name=context.dataset_info.predictions_table,
                index=False,
                if_exists="append")

    print("Saved predictions in Teradata")
    
    # calculate stats
    predictions_df = DataFrame.from_query(f"""
        SELECT 
            * 
        FROM {context.dataset_info.get_predictions_metadata_fqtn()} 
            WHERE job_id = '{context.job_id}'
    """)

    record_scoring_stats(features_df=DataFrame.from_query(context.dataset_info.sql),
                         predicted_df=predictions_df,
                         context=context)

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the scoring dataset 

sql = """
SELECT 
    F.*
FROM Demo_ModelOps.PIMA_PATIENT_FEATURES F 
    WHERE F.patientid MOD 5 = 0
"""

# where to store predictions
predictions = {
    "database": "demo_user",
    "table": "pima_patient_predictions_tmp"
}

import uuid
job_id=str(uuid.uuid4())

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata,
                           predictions=predictions)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="artifacts/",
                   artifact_input_path="artifacts/",
                   model_version="h2oaml_v1",
                   model_table="model_h2oaml_v1",
                   job_id=job_id)

# drop volatile table from session if executing multiple times
try:
    db_drop_table('byom_models_tmp')
except: 
    pass

import scoring
scoring.score(context=ctx)

In [ ]:
DataFrame.from_query(f"SELECT * FROM Demo_User.pima_patient_predictions_tmp WHERE job_id='{job_id}'")

In [ ]:
# Clean up

os.system('rm -f artifacts/*')

try:
    db_drop_table('model_h2oaml_v1')
except: 
    pass

try:
    db_drop_table('pima_patient_predictions_tmp')
except: 
    pass

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Define Model Metadata</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let's create the configuration files.<br>Requirements file with the dependencies and versions:</p>

In [ ]:
%%writefile $model_local_path/model_modules/requirements.txt
teradatamodelops==7.0.3
teradataml==17.20.0.6
pandas==2.1.4
matplotlib==3.8.2
scikit-learn==1.3.2
h2o==3.44.0.3
install-jdk==1.0.4

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The hyper parameter configuration (default values):</p>

In [ ]:
%%writefile $model_local_path/config.json
{
   "hyperParameters": {
      "max_models": 10,
      "seed": 1
   }
}

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The model configuration:</p>

In [ ]:
%%writefile $model_local_path/model.json
{
    "id": "cdce3e32-3f8c-417f-b10e-493f21740a92",
    "name": "Python PIMA H2O AutoML",
    "description": "Python PIMA H2O AutoML for Diabetes Prediction",
    "language": "python"
}

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Commit and Push to Git to let ModelOps manage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Run the command below to commit and push changes to our forked repository, so ModelOps can fetch the changes to the model.</p>

In [ ]:
!cd $model_local_path/../.. && git add . && git commit -m "Added Python PIMA H2O AutoML demo model 💦" && git push

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now that changes are pushed, you can make the lifecycle inside <b>ModelOps User Interface</b>, plan for new trainings, evaluations, scorings. Compare models and operationalize into Production with automated Monitoring and alerting capabilities.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>8. ModelOps full lifecycle till deployment</b></p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use or Create a Project with the git code repository with the model code, then you should see the model in the catalog already created</p>

<img src="images/08_01.png" alt="Model Catalog with inDB"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Select the Model and then click Train a new Model. Use default hyper-parameters. This will launch the training job with the training script we generated and pushed to Git.</p>

<img src="images/08_02.png" alt="Train"/>

<img src="images/08_03.png" alt="Train job" width="500" height="500"/>

<img src="images/08_04.png" alt="Train finished" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>When Model is trained a new Model Id is created and you can get inside the Model Lifecycle screen to review artifacts and other details</p>

<img src="images/08_06.png" alt="Model lifecycle"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, let's evaluate the Model, click the button and select the evaluation dataset. This will launch the evaluation job with the training script we generated and pushed to Git.</p>

<img src="images/08_07.png" alt="Evaluation" width="500" height="500"/> <img src="images/08_08.png" alt="Evaluation job" width="500" height="500"/>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>When evaluation job is finished a Model evaluation Report is generated with the metrics and charts that evaluation script generates</p>

<img src="images/08_26.png" alt="Model Report" />

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now, let's approve the model and provide an approval description</p>

<img src="images/08_09.png" alt="Approval" />

<img src="images/08_10.png" alt="Approval description" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The model is ready to be deployed. Let's deploy using a Batch scheduling option - Run it manual</p>

<img src="images/08_11.png" alt="Deployment Engine" width="500" height="500"/>

<img src="images/08_12.png" alt="Deployment Publish" width="500" height="500"/>

<img src="images/08_13.png" alt="Deployment Schedule" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

[![image](images/launchModelOps.png)](/modelops)

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>9. ModelOps Monitoring</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now the model is deployed and a new Deployment appears in the deployment screen</p>


<img src="images/08_15.png" alt="Deploymet" />


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You can run jobs manually from here, review history of executions and view the predictions for a specific job</p>

<img src="images/08_16.png" alt="Deployment Run" width="500" height="500"/>

<img src="images/08_17.png" alt="Deployment Jobs" />

<img src="images/08_18.png" alt="Deployment view" width="500" height="500" />

<img src="images/08_19.png" alt="Deployment predictions" width="500" height="500"/>

<img src="images/08_20.png" alt="Deployment" width="500" height="500"/>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the Feature Drift and Prediction Drift tabs you can check on the monitoring of the data drift</p>

<img src="images/08_22.png" alt="Feature Drift" />

<img src="images/08_21.png" alt="Prediction Drift" />

<img src="images/08_23.png" alt="Performance Monitoring" />



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the Performance Drift, you can review multiple evaluations, let's evaluate the model with a new dataset. We create a new evaluation dataset with this query:</p>
    
```sql
SELECT * FROM pima_patient_diagnoses F WHERE F.patientid MOD 8 <> 0  
```

<img src="images/08_24.png" alt="Evaluate" width="500" height="500" />

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>and now see the evolution of the metrics</p>

<img src="images/08_25.png" alt="Metrics monitoring" />

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
With ModelOps you can close the cycle and review make decisions when you need to replace yor model in production, For example, You could get alerting from Data Drift of Performance Drift and you can create multiple versions and compare them, select a champion and deploy new versions that replace existing in Production.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

[![image](images/launchModelOps.png)](/modelops)

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>10. Cleanup</b></p>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If you are done with ModelOps usecase, please uncomment and run the below cleanup section.</p>
</div>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
# db_drop_table(table_name = 'aoa_byom_models', schema_name = 'demo_user')
# db_drop_table(table_name = 'pima_patient_predictions', schema_name = 'demo_user')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
# %run -i ../UseCases/run_procedure.py "call remove_data('DEMO_ModelOps');"        # Takes 10 seconds

In [ ]:
remove_context()

[<< Back to GIT PIMA H2OAutoML](./08_ModelOps_GIT_PIMA_Python_H2OAutoML.ipynb) | [Continue to GIT PIMA XGBoost >>](./09_ModelOps_GIT_PIMA_Python_XGboost.ipynb)

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>